# Optimizing the sentiment agent model

Optimizing the parameters of the functions in the sentiment agent model. This is a work in progress.

Written by Luc Bijl

Importing the posts dataset.

In [ ]:
import pandas as pd

string_start = '2023-10-07 06:30'
string_end = '2023-10-07 13:30'
import_posts = '../datasets/ishw-case/scored/israel-scored.csv'
resample_period = '10min'

df_posts = pd.read_csv(import_posts)
df_posts['date'] = pd.to_datetime(df_posts['date'])

Defining the performance metric and objective function used in the optimization.

In [ ]:
import numpy as np
from scripts.sentiment_agent_model import sentiment_agent_model
from functools import reduce
import operator

# Defining the performance metric function.
def performance_metric(df_posts, df_modeled_posts, string_start, string_end, resample_period):
    df_posts_date = df_posts.set_index('date').sort_index().loc[string_start:string_end]
    df_modeled_posts_date = df_modeled_posts.set_index('date').sort_index().loc[string_start:string_end]

    posts_resampled = {'s': [], 'v': [], 'a': [], 'd': []}
    posts_model_resampled = {'s': [], 'v': [], 'a': [], 'd': []}
    
    punishment = 100

    for i in ['s', 'v', 'a', 'd']:
        posts_resampled[i] = df_posts_date[i].resample(resample_period).mean().dropna()
        posts_model_resampled[i] = df_modeled_posts_date[i].resample(resample_period).mean().fillna(punishment)

    difference = reduce(operator.add, [abs(posts_resampled[i] - posts_model_resampled[i]) for i in ['s', 'v', 'a', 'd']]).fillna(punishment)

    return sum(difference)

# Defining the objective function.
def objective_function(params, string_start, string_end, import_posts):
    model = {}

    # Updating the model parameters with the optimized values.
    model['sp'] = {'age': params[0], 
               's': [[params[1], params[2], params[3]], [params[4], params[5], params[6]]],
               'v': [[params[7], params[8], params[9]], [params[10], params[11], params[12]]],
               'a': [[params[13], params[14], params[15]]],
               'd': [[params[16], params[17], params[18]]]}
    model['cif'] = [[params[19], params[20], params[21]]]
    model['pp'] = {'s': [[params[22], params[23], params[24]], [params[25], params[26], params[27]]],
               'v': [[params[28], params[29], params[30]], [params[31], params[32], params[33]]],
               'a': [[params[34], params[35], params[36]]],
               'd': [[params[37], params[38], params[39]], [params[40], params[41], params[42]]]}

    cardinality_S = int(params[43])
    post_period = int(params[44])

    # Running the sentiment agent model and returning the performance metric.
    df_modeled_posts = sentiment_agent_model(import_posts, cardinality_S, string_start, string_end, post_period, model)
    metric_to_optimize = performance_metric(df_posts, df_modeled_posts, string_start, string_end, resample_period)
    print(metric_to_optimize)

    return metric_to_optimize

Defining the optimization method and its necessary conditions and bounds.

In [ ]:
from scipy.optimize import differential_evolution, Bounds

# Defining the bounds for each parameter.
lower_bounds = [-20] * 45
upper_bounds = [20] * 45

lower_bounds[0] = 0
upper_bounds[0] = 1
lower_bounds[43] = 10
upper_bounds[43] = 40
lower_bounds[44] = 10
upper_bounds[44] = 1000

bounds = Bounds(lower_bounds, upper_bounds)

# Defining the initial guess for each parameter.
initial_guess = [0.1, 
                0.4, 1.5, 1,
                0.6, -1.5, -1,
                0.4, 1.5, 1,
                0.6, -1.5, -1,
                1, 2, 0,
                1, 0.5, 0,
                1, -1.5, 3,
                0.8, 7, 0.05, 
                0.8, -7, -0.05, 
                0.8, 7, 0.05, 
                0.8, -7, -0.05,
                1, 2, -1,
                0.8, 7, 0.05, 
                0.8, -7, -0.05,
                30, 60]

# Running the optimization.
result = differential_evolution(objective_function, bounds, args=(string_start, string_end, import_posts), strategy='best1bin', maxiter=100)

# Obtaining the optimized parameters.
optimized_params = result.x
print("Optimized parameters:", optimized_params)